In [1]:
import pandas as pd
import numpy as np
import os
import time
from IPython.display import clear_output

In [2]:
root = '/home/MIBS/facenet'
target_folder = os.path.join(root, 'embedding', 'similarity')

## Read data

In [3]:
df_sim = pd.read_csv(os.path.join(target_folder, 'similarity_0.csv'), index_col=0)
df_sim.head()

,1000046113_190136.jpg,1000046113_190137.jpg,1000046113_190138.jpg,1000046113_190139.jpg,1000046113_190140.jpg
1000046113_190136.jpg,0.000000,0.043208,0.059137,0.077802,0.084179
1000046113_190137.jpg,0.043208,0.000000,0.015239,0.032346,0.046843
1000046113_190138.jpg,0.059137,0.015239,0.000000,0.011081,0.033390
1000046113_190139.jpg,0.077802,0.032346,0.011081,-0.000000,0.017354
1000046113_190140.jpg,0.084179,0.046843,0.033390,0.017354,-0.000000


In [4]:
df_sim

,1000046113_190136.jpg,1000046113_190137.jpg,1000046113_190138.jpg,1000046113_190139.jpg,1000046113_190140.jpg
1000046113_190136.jpg,0.000000,0.043208,0.059137,0.077802,0.084179
1000046113_190137.jpg,0.043208,0.000000,0.015239,0.032346,0.046843
1000046113_190138.jpg,0.059137,0.015239,0.000000,0.011081,0.033390
1000046113_190139.jpg,0.077802,0.032346,0.011081,-0.000000,0.017354
1000046113_190140.jpg,0.084179,0.046843,0.033390,0.017354,-0.000000
...,...,...,...,...,...
9876_10089.jpg,0.898189,0.889843,0.867038,0.847671,0.842859
9876_10090.jpg,0.884373,0.877761,0.853807,0.831718,0.824670
9876_10091.jpg,0.865650,0.857599,0.833253,0.810912,0.804445
9876_10092.jpg,0.887948,0.880207,0.856261,0.833607,0.825441


## Check similarity values

In [62]:
similarity_data = []
threshold = .25

tp, tn, fp, fn = 0, 0, 0, 0

for column, data_column in df_sim.items():
    idx_1 = column.split('.')[0].split('_')[0]
    tmp_dict = {column: {'same': {'min': None, 'max': None}, 'different': {'min': None, 'max': None}}}
    for row, value in data_column.items():
        idx_2 = row.split('.')[0].split('_')[0]
        if idx_1 == idx_2:
            # same person
            if value < threshold:
                tp += 1
            else:
                fn += 1
            current_min = tmp_dict[column]['same']['min']
            current_max = tmp_dict[column]['same']['max']
            if current_min == None or current_max == None:
                tmp_dict[column]['same']['min'] = value
                tmp_dict[column]['same']['max'] = value
            else:
                if value < tmp_dict[column]['same']['min']:
                    tmp_dict[column]['same']['min'] = value
                if value > tmp_dict[column]['same']['max']:
                    tmp_dict[column]['same']['max'] = value
        else:
            # different person
            if value < threshold:
                fp += 1
            else:
                tn += 1
            current_min = tmp_dict[column]['different']['min']
            current_max = tmp_dict[column]['different']['max']
            if current_min == None or current_max == None:
                tmp_dict[column]['different']['min'] = value
                tmp_dict[column]['different']['max'] = value
            else:
                if value < tmp_dict[column]['different']['min']:
                    tmp_dict[column]['different']['min'] = value
                if value > tmp_dict[column]['different']['max']:
                    tmp_dict[column]['different']['max'] = value
    similarity_data.append(tmp_dict)

In [63]:
print('TP: {}, TN: {}, FP: {}, FN: {}'.format(tp, tn, fp, fn))

TP: 2047, TN: 2323558, FP: 2, FN: 458


In [64]:
precision = float(tp) / (tp + fp)
recall = float(tp) / (tp + fn)
f1 = float(2 * precision * recall) / (precision + recall)

print('Precision: {} - Recall: {} - F1: {}'.format(precision, recall, f1))

Precision: 0.9990239141044412 - Recall: 0.8171656686626747 - F1: 0.8989898989898991


In [43]:
similarity_data

[{'1000046113_190136.jpg': {'same': {'min': 0.0, 'max': 0.470012},
   'different': {'min': 0.233462, 'max': 1.305814}}},
 {'1000046113_190137.jpg': {'same': {'min': 0.0, 'max': 0.500817},
   'different': {'min': 0.3018, 'max': 1.360845}}},
 {'1000046113_190138.jpg': {'same': {'min': 0.0, 'max': 0.52361},
   'different': {'min': 0.297147, 'max': 1.386028}}},
 {'1000046113_190139.jpg': {'same': {'min': -0.0, 'max': 0.558896},
   'different': {'min': 0.27129699999999995, 'max': 1.373637}}},
 {'1000046113_190140.jpg': {'same': {'min': -0.0, 'max': 0.584184},
   'different': {'min': 0.24246900000000002, 'max': 1.368933}}}]

In [36]:
sd_1 = {idx_1: {'same': {'min': None, 'max': None}, 'different': {'min': None, 'max': None}}}
sd_1

{'888': {'same': {'min': None, 'max': None},
  'different': {'min': None, 'max': None}}}

In [37]:
reform = {(outerKey, innerKey): values for outerKey, innerDict in sd_1.items() for innerKey, values in innerDict.items()}


In [31]:
tmp = pd.DataFrame(reform).T
tmp

min   max
1000046113 same       None  None
           different  None  None

In [38]:
pd.concat([tmp, pd.DataFrame(reform).T])

min   max
1000046113 same       None  None
           different  None  None
888        same       None  None
           different  None  None